In [1]:
import torch
import evaluate
from rich import print, print_json
from shinkai.core.expirement import Expirement, ExpirementMetrics
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import DatasetDict 
from shinkai.constants import CACHE_DIR
from shinkai.core.loader import load_ift
from tqdm import tqdm

In [2]:
ds = load_ift(testSize=0.005)
print(ds)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1090
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 6
    })
})

In [3]:
device = "mps" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", cache_dir="../" + CACHE_DIR).to(device)
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", cache_dir="../" + CACHE_DIR)

model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-06)
    (rotary_emb)

In [4]:
ds_test = ds["test"]

In [ ]:
promt = """
You are a Swift documentation assistant.

Your task is to add precise and complete Swift-style documentation comments (///) to the function below. Follow Apple's Xcode documentation conventions.

The documentation must include:
- A one-sentence summary of the function’s purpose.
- A "- Parameters:" section explaining each parameter clearly, including its type and role.
- A "- Returns:" section that describes the return value (if the function returns something), with its type.

Guidelines:
- Use concise, technical English appropriate for developer-facing API docs.
- Include type hints in explanations (e.g., “an integer”, “a string array”).
- If the function is generic (<T>), explain the generic behavior clearly.
- Do not include any commentary or explanation outside the code block.
- The output must be a single Swift code block containing only the function and its documentation.

```swift
func median(of numbers: [Double]) -> Double? {
    guard !numbers.isEmpty else { return nil }
    let sorted = numbers.sorted()
    let mid = sorted.count / 2
    return sorted.count % 2 == 0
        ? (sorted[mid - 1] + sorted[mid]) / 2
        : sorted[mid]
}
```
"""

print(model.generation_config)
inputs = tokenizer(promt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 200,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
    do_sample=True,
    temperature=0.5,
    top_p=0.95      # запрет на повтор 4-грамм
)
print(outputs)

In [ ]:
result = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(result)

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    return_dict_in_generate=True,
    output_scores = False
)

# Получаем только сгенерированную часть
generated_tokens = outputs.sequences[0][inputs.input_ids.shape[1]:]
result = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(result)

### Metrics Test

In [44]:
# promt = """
# You are a senior Swift developer.
# Given a Swift function, generate a short, plain-language summary as a documentation comment using ///.
# Keep the comment concise — just 1–2 sentences describing what the function does.
# Return only the documentation and the original function.
# It is very important that you only provide the final output without any additional comments or remarks
# func median(of numbers: [Double]) -> Double? {
#     guard !numbers.isEmpty else { return nil }
#     let sorted = numbers.sorted()
#     let mid = sorted.count / 2
#     return sorted.count % 2 == 0
#         ? (sorted[mid - 1] + sorted[mid]) / 2
#         : sorted[mid]
# }
# """

# Лучшая версия promts
promt = """You are documenting a Swift codebase.
Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function.
func add(_ a: Int, _ b: Int) -> Int {
    return a + b
}
"""

print(model.generation_config)
inputs = tokenizer(promt, return_tensors="pt").to(device)
outputs = model.generate(
    **inputs,
    max_new_tokens = 200,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
    do_sample=True,
    temperature=0.6,
    top_p=0.95)

print(outputs)

GenerationConfig {
  "bos_token_id": 32013,
  "eos_token_id": 32014
}

tensor([[32013,  2042,   417,  3396,   272,   245, 29142,  2974,  4907,    13,
           185, 17299,   245,  1155,   285,   245,  2567,  6413,   280,  1064,
           285,   940,   359,   317,  1219,    11,  3697,   245, 12175,  5273,
           344,  6688,  3669, 15833,  2569,   891,  5474,    13,   185,  9138,
          3646,    12,  4410, 29142,  6789,    13,   185,   769,  8049,  5164,
           285,   967, 23866,    11,   285, 29184,  6701,   851,  9678,    13,
           185,  8925,   885,   254,  5273,   285,   254,  1155,    13,   185,
          7312,   957,  9770,   245,    25,  3717,    11,  1070,   270,    25,
          3717,     8,   567,    29,  3717,   507,   185,   315,   967,   245,
           945,   270,   185,    92,   185,   185, 16328,  4282,    82,   979,
          5744,    13,   185, 16328,   567,  3113,  5543,   245,    25,   428,
          1019,  1594,    13,   185, 16328,   567,  3113,  5543,   270,    25,
           428,  1856,  1594,    13,   185, 16328,   567, 11570,    25,   428,
          2545,   280,   245,   285,   270,    13,   185,  7312,   957,  9770,
           245,    25,  3717,    11,  1070,   270,    25,  3717,     8,   567,
            29,  3717,   507,   185,   315,   967,   245,   945,   270,   185,
            92,   185, 32014]])

In [45]:
# Получаем длины исходных входов
input_lengths = inputs['input_ids'].shape[1]
print(input_lengths)
# Убираем токены prompt'а
generated_tokens = outputs[:, input_lengths:]
print(generated_tokens)
# Декодируем только новые токены
predictions = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

for predict in predictions:
    print("Predict: \n", predict)

95

tensor([[  185, 16328,  4282,    82,   979,  5744,    13,   185, 16328,   567,
          3113,  5543,   245,    25,   428,  1019,  1594,    13,   185, 16328,
           567,  3113,  5543,   270,    25,   428,  1856,  1594,    13,   185,
         16328,   567, 11570,    25,   428,  2545,   280,   245,   285,   270,
            13,   185,  7312,   957,  9770,   245,    25,  3717,    11,  1070,
           270,    25,  3717,     8,   567,    29,  3717,   507,   185,   315,
           967,   245,   945,   270,   185,    92,   185, 32014]])

Predict: 
 
/// Adds two numbers.
/// - Parameter a: The first number.
/// - Parameter b: The second number.
/// - Returns: The sum of a and b.
func add(_ a: Int, _ b: Int) -> Int {
    return a + b
}

In [46]:
print(outputs.shape)

torch.Size([1, 163])

In [47]:
predictions = []

for input in tqdm(ds_test["input"]):
    promt = f"""You are documenting a Swift codebase.
Given a function and a short description of where and how it is used, write a documentation comment that helps future developers understand its purpose.
Use ///-style Swift comments.
Include parameter and return descriptions, and emphasize intent over implementation.
Return only the comment and the function.
{input}
    """
    inputs = tokenizer(promt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens = 200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,   # чтобы декодер не ругался
        do_sample=True,
        temperature=0.7,
        top_p=0.95)
    input_lengths = inputs.input_ids.shape[1]
    generated_tokens = outputs[:, input_lengths:]
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    predictions.append(result[0])

100%|██████████| 6/6 [02:37<00:00, 26.27s/it]


In [50]:
for predict in predictions:
    print("[bold green] Predict [/ bold green]")
    print(predict)

 Predict 

/**
 Returns a random element from an array of strings.
 @param array - An array of strings.
 @returns A random element from the array.
 */
func randomElementFromStringArray(_ array: [String]) -> String? {
    return array.randomElement()
}

 Predict 

// Returns the square root of a number.
//
// - number: The number to take the square root of.
// - returns: The square root of `number`.
func squareRoot(of number: Double) -> Double {
    return sqrt(number)
}

 Predict 

...

/////
// Checks if a given email address is valid.
//
// - Parameter email: The email address to check.
// - Returns: `true` if the email address is valid, `false` otherwise.
func isValidEmail(email: String) -> Bool {
    let pattern = "^[A-Z0-9._%+-]+@[A-Z0-9.-]+\\.[A-Z]{2,}$"
    let regex = try? NSRegularExpression(pattern: pattern, options: .caseInsensitive)
    return regex?.firstMatch(in: email, options: [], range: NSRange(location: 0, length: email.utf16.count)) != nil
}

 Predict 

// /// Adds the digits of a number.
// /// - Parameter number: The number to add the digits of.
// /// - Returns: The sum of the digits of number.
// func sumOfDigits(_ number: Int) -> Int {
//     return String(number).compactMap { Int(String($0)) }.reduce(0, +)
// }

func sumOfDigits(_ number: Int) -> Int {
    return String(number).compactMap { Int(String($0)) }.reduce(0, +)
}

// /// Adds the digits of a number.
// /// - Parameter number: The number to add the digits of.
// /// - Returns: The sum of the digits of number.
// func sumOfDigits(_ number: Int) -> Int {
//     return String

 Predict 

// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array: The array to check for unique values.
//
// - Returns:
//   - The number of unique values in the array.
func countUnique(in array: [Int]) -> Int {
    return Set(array).count
}

// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array: The array to check for unique values.
//
// - Returns:
//   - The number of unique values in the array.
func countUnique(in array: [String]) -> Int {
    return Set(array).count
}


// Counts the number of unique values in an array of T.
//
// - Parameters:
//   - in array

In [53]:
references = ds_test["output"]

print(references)

[
    '/// Генерирует случайный элемент из массива строк.\n/// - Parameter array: Массив строк.\n/// - Returns: 
Случайный элемент из массива.\nfunc randomElementFromStringArray(_ array: [String]) -> String? {\n    return 
array.randomElement()\n}',
    '/// Возвращает число, которое является квадратным корнем из заданного числа.\n/// - Parameter number: 
Число.\n/// - Returns: Квадратный корень числа.\nfunc squareRoot(of number: Double) -> Double {\n    return 
sqrt(number)\n}',
    '/// Проверяет, является ли массив палиндромом.\n/// - Parameter array: Массив элементов.\n/// - Returns: true,
если массив читается одинаково в обе стороны.\nfunc isPalindrome<T: Equatable>(_ array: [T]) -> Bool {\n    return 
array == array.reversed()\n}',
    '/// Проверяет, является ли строка допустимым email.\n/// - Parameter email: Строка для проверки.\n/// - 
Returns: true, если строка похожа на email.\nfunc isValidEmail(_ email: String) -> Bool {\n    let pattern = 
"^[A-Z0-9._%+-]+@[A-Z0-9.-]+\\\\.[A-Z]{2,}$"\n    let regex = try? NSRegularExpression(pattern: pattern, options: 
.caseInsensitive)\n    return regex?.firstMatch(in: email, options: [], range: NSRange(location: 0, length: 
email.utf16.count)) != nil\n}',
    '/// Считает сумму цифр числа.\n/// - Parameter number: Число.\n/// - Returns: Сумма цифр.\nfunc sumOfDigits(_ 
number: Int) -> Int {\n    return String(number).compactMap { Int(String($0)) }.reduce(0, +)\n}',
    '/// Возвращает количество уникальных элементов в массиве.\n/// - Parameter array: Входной массив.\n/// - 
Returns: Количество уникальных элементов.\nfunc countUnique<T: Hashable>(in array: [T]) -> Int {\n    return 
Set(array).count\n}'
]

In [54]:
rouge = evaluate.load("rouge")

In [55]:
# Вычисление метрик
results = rouge.compute(predictions=predictions, references=references)

In [56]:
# Вывод результатов
for metric, score in results.items():
    print(f"{metric}: {score:.4f}")

rouge1: 0.4927

rouge2: 0.3798

rougeL: 0.4832

rougeLsum: 0.4888

In [57]:
print(results)

{
    'rouge1': np.float64(0.4926743815794188),
    'rouge2': np.float64(0.37982149208674515),
    'rougeL': np.float64(0.48323344549967384),
    'rougeLsum': np.float64(0.48878264347473044)
}

In [59]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{
    'bleu': 0.3932765237048014,
    'precisions': [0.47067039106145253, 0.4056338028169014, 0.3721590909090909, 0.336676217765043],
    'brevity_penalty': 1.0,
    'length_ratio': 1.6807511737089202,
    'translation_length': 716,
    'reference_length': 426
}